In [1]:
import torch
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

## **Task**

We want to train an MLP to learn the construction of names. To achieve this we want to maximise the likelihood of the $N$ names observed in the dataset:

$$
\max_{\theta} \sum_{i=1}^{N} \hat{p}(\mathbf{x}_i) = \max_{\theta} \sum_{i=1}^{N} \prod_{j=1}^{M_i} \hat{p}(x_j | x_{j-1}, x_{j-2}, x_{j-3}; \theta)
$$

In particular, we choose to break down the probability of a name into the product of the conditional probabilities of each character given a three-letter context window (we use '.' as the special token to pad the start and end of a name). For example, the name "John" would give us the following data points:

* ... $\rightarrow$ J
* ..J $\rightarrow$ o
* .Jo $\rightarrow$ h
* Joh $\rightarrow$ n
* ohn $\rightarrow$ .

We begin by constructing a training set of such data points from all the names in the dataset. We then use the negative log-likelihood of the data as the loss function to be minimized. See `mm_intro.ipynb` for more details.

# **Dataset**

In [2]:
# Load data
words = open('names.txt', 'r').read().splitlines()
print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [3]:
# Create a dictionary that maps characters to integers and vice versa
char2idx = {c: i+1 for i, c in enumerate('abcdefghijklmnopqrstuvwxyz')}
char2idx['.'] = 0 # special character for marking start and end of a word
idx2char = {i: c for c, i in char2idx.items()}

In [4]:
# Form training pairs (context, target characters)
block_size = 3 # context size for next character prediction

def build_dataset(words):
    X, Y = [], []
    for word in words:
        w2idx = [0] * block_size + [char2idx[c] for c in word] + [0]
        for i in range(len(w2idx) - block_size):
            X.append(w2idx[i:i+block_size])
            Y.append(w2idx[i+block_size])

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape, X.dtype, Y.dtype)
    return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(len(words)*0.8)
n2 = int(len(words)*0.9)

X_train, Y_train = build_dataset(words[:n1]) # 80% of words
X_val, Y_val = build_dataset(words[n1:n2]) # 10% of words
X_test, Y_test = build_dataset(words[n2:]) # 10% of words

# print first 10 samples of X_train and Y_train
for i in range(len(Y_train[:10])):
    print(''.join([idx2char[idx.item()] for idx in X_train[i]]), '->', idx2char[Y_train[i].item()])

torch.Size([182625, 3]) torch.Size([182625]) torch.int64 torch.int64
torch.Size([22655, 3]) torch.Size([22655]) torch.int64 torch.int64
torch.Size([22866, 3]) torch.Size([22866]) torch.int64 torch.int64
... -> y
..y -> u
.yu -> h
yuh -> e
uhe -> n
hen -> g
eng -> .
... -> d
..d -> i
.di -> o


# **Model**

We begin by implementing PyTorch-like modules.

In [5]:
class Embedding:
    def __init__(self, num_embeddings, emb_dim):
        self.W = torch.randn((num_embeddings, emb_dim)) # / dim**0.5

    def __call__(self, x):
        self.out = self.W[x]
        return self.out
    
    def parameters(self):
        return [self.W]

class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.W = torch.randn((fan_in, fan_out)) / fan_in**0.5
        self.b = torch.zeros((fan_out)) if bias else None

    def __call__(self, x):
        self.out = x @ self.W
        if self.b is not None:
            self.out += self.b
        return self.out
    
    def parameters(self):
        if self.b is not None:
            return [self.W, self.b]
        else:
            return [self.W]
        
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.gain = torch.ones((dim))
        self.bias = torch.zeros((dim))
        self.eps = eps
        self.training = True
        self.momentum = momentum
        self.mean_running = torch.zeros((dim))
        self.var_running = torch.ones((dim))

    def __call__(self, x):
        if self.training:
            x_mean = x.mean(dim=0, keepdim=True)
            x_var = x.var(dim=0, keepdim=True)
        else:
            x_mean = self.mean_running
            x_var = self.var_running
        x_hat = (x - x_mean) / (x_var + self.eps).sqrt()
        self.out = self.gain * x_hat + self.bias
        if self.training:
            with torch.no_grad():
                self.mean_running = (1 - self.momentum) * self.mean_running + self.momentum * x_mean
                self.var_running = (1 - self.momentum) * self.var_running + self.momentum * x_var
        return self.out
    
    def parameters(self):
        return [self.gain, self.bias]
    
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []
    
class Flatten:
    def __call__(self, x):
        self.out = x.view(x.shape[0], -1)
        return self.out
    
    def parameters(self):
        return []
    
class Sequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        for l in self.layers:
            x = l(x)
        self.out = x
        return self.out
    
    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

In [6]:
torch.manual_seed(42); # seed RNG for reproducibility

In [7]:
emb_dim = 10 # embedding dimension
h_dim = 200 # hidden layer dimension

model = Sequential([
    Embedding(27, emb_dim), Flatten(),
    Linear(block_size * emb_dim, h_dim, bias=False), BatchNorm1d(h_dim), Tanh(), 
    Linear(h_dim, 27)
])

with torch.no_grad():
    model.layers[-1].W *= 0.1 # make last linear layer less confident (more uniform) predictions

parameters = model.parameters()
print('Number of parameters:', sum([p.numel() for p in parameters]))

# Always perform last 
for p in parameters:
    p.requires_grad = True

Number of parameters: 12097


## Training

In [8]:
# Training parameters
max_steps = 200000
batch_size = 32
loss_i = []
upd2data = []

In [9]:
# Training loop
for i in range(max_steps):
    # mini-batch sampling
    batch_idxs = torch.randperm(len(Y_train))[:batch_size]
    X_batch, Y_batch = X_train[batch_idxs], Y_train[batch_idxs]
  
    # forward pass
    logits = model(X_batch)
    loss = F.cross_entropy(logits, Y_batch)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update parameters
    lr = 0.1 if i < 150000 else 0.01 # decrease learning rate after 150K steps
    for p in parameters:
        p.data -= lr * p.grad

    # track stats
    if i % 1000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    loss_i.append(loss.log10().item())
    # with torch.no_grad():
    #     upd2data.append([(lr * p.grad.std() / p.data.std()).log10().item() for p in parameters])

      0/ 200000: 3.3121
   1000/ 200000: 2.6946
   2000/ 200000: 2.4040


In [10]:
# We smoothen out the loss curve by averaging over 1000 steps
# Due to the small batch size of 32, it would be too noisy otherwise
plt.plot(torch.tensor(loss_i).view(-1, 1000).mean(1))

RuntimeError: shape '[-1, 1000]' is invalid for input of size 2001

## Validation

In [ ]:
# put layers into eval mode (needed for batch norm especially)
for l in model.layers:
    l.training = False

In [11]:
# forward pass for loss validation
@torch.no_grad() # this decorator disables gradient tracking
def split_losss(split):
    x, y = {
        'train': (X_train, Y_train),
        'val': (X_val, Y_val),
        'test': (X_test, Y_test)
    }[split] 
    logits = model(x)
    loss = F.cross_entropy(logits, y) 
    print('Loss:', loss.item())

split_losss('train')
split_losss('val')

Loss: 2.372546434402466
Loss: 2.3663876056671143


## Inference

In [12]:
for _ in range(10):
    name = ''
    context = [0] * block_size
    while True:
        # forward pass
        logits = model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)
        # sample next character
        i = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [i]
        name += idx2char[i]
        if i == 0: 
            break
    print(name)

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

## Analysing Activations

Now, we visualize both the activations and their gradients of the hidden layers after one iteration to see if there are any issues. The main thing to look out for is that our activations are not overly saturated or too inactive. Furthermore, if the gradients through the activation nodes are too small or too big, then the model will not be able to learn effectively either. Overall, it's important to have a similar distribution of activations and gradients across all layers of the model to allow for optimal gradient flow and learning. If the distribution of activations and gradients changes significantly across the layers, i.e. there is poor congruence across their distributions, then we must investigate whether our initializations are appropriate or whether we need to introduce some form of normalization such as BatchNorm.

In [ ]:
# Plot activations 
plt.figure(figsize=(20, 4))
legends = []
for i, l in enumerate(layers):
    if isinstance(l, Tanh):
        t = l.out
        print('layer %d (%10s): mean %.2f, std %.2f, saturated: %.2f%%' % (i, l.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean() * 100))
        hy, hx = torch.histogram(t, density=True)
        plt.plot(hx[:-1].detach(), hy.detach())
        legends.append(f'layer {i} ({l.__class__.__name__})')
plt.legend(legends);
plt.title('activation distribution');

In [ ]:
# Plot activation gradients
plt.figure(figsize=(20, 4))
legends = []
for i, l in enumerate(layers):
    if isinstance(l, Tanh):
        t = l.out.grad
        print('layer %d (%10s): mean %+f, std %e' % (i, l.__class__.__name__, t.mean(), t.std()))
        hy, hx = torch.histogram(t, density=True)
        plt.plot(hx[:-1].detach(), hy.detach())
        legends.append(f'layer {i} ({l.__class__.__name__})')
plt.legend(legends);
plt.title('gradient distribution');

## Analysing Weights 

In addition to analysing the activations and their gradients, we also want to analyse the weights and visualize their distribution. Similar to the activations, we want to see a similar distribution of weights across all layers of the model. If certain weights are too large relative to others, then the gradients flowing through these weights will also be large and cause these areas to learn faster than others (assuming a simple optimzation approach such as SGD). This may lead to the model getting stuck in a local minimum. 

Finally, we also study the gradient-to-weight ratio ("grad:data ratio") which is a measure of how much the weights change in response to a change in the loss. If the ratio is too large, then the weights will change too much and the model will not be able to learn effectively. If the ratio is too small, then the weights will not change enough and the model will also not be able to learn effectively. Naturally, this also depends on the learning rate so we must put this into context, which is why we create the second plot below showing the actual _update to weight ratio_ which takes the learning rate into account.

In [ ]:
# Plot weight gradient distribution
plt.figure(figsize=(20, 4))
legends = []
for i, p in enumerate(parameters):
    t = p.grad
    if p.ndim == 2:
        # Note that we compute the grad:data ration using the std, not the mean as the mean may be close to zero 
        # wheras the std gives us a good idea of the scale of the gradients and the data
        print('weight %10s | mean %+f | std %e | grad:data ratio %e' % (tuple(p.shape), t.mean(), t.std(), t.std() / p.std()))
        hy, hx = torch.histogram(t, density=True)
        plt.plot(hx[:-1].detach(), hy.detach())
        legends.append(f'{i} {tuple(p.shape)}')
plt.legend(legends);
plt.title('weights gradient distribution');

### Update to Weight Ratio

In general, our _update to weight ratios_ for each of the layers look good and lie around the "optimal" value of $1\mathrm{e}{-3}$. However, we do observe a slight outlier in the the output layer which has a much larger ratio. This is because we scaled the weights of the output layer by a factor of $0.1$ in order to bring the initialisation of the weights closer to zero and hence reduce the random overconfidence of the model for one particular character (essentially bringing it closer to uniform distribution). We observe that this outlier stabilizes after a few iterations and the ratio is brought down to a more reasonable value.

In [ ]:
# Plot update to weight (data) ratio
plt.figure(figsize=(20, 4))
legends = []
for i, p in enumerate(parameters):
    if p.ndim == 2:
        plt.plot([upd2data[j][i] for j in range(len(upd2data))])
        legends.append('param %d' % i)
plt.plot([0, len(upd2data)], [-3, -3], 'k') # guideline, the ratios should be ~1e-3
plt.legend(legends);